In [1]:
import pandas as pd
import os
import numpy as np
import html
import re
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

/home/charlie/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


# Settings

In [2]:
MAX_LEN = 512
BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased', do_lower_case=True) # Need to retrain with do_lower_case=False
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=1).to(device)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifi

# Preprocessing

In [3]:
DATA_DIR = '../data/raw/'
filenames = os.listdir(DATA_DIR)
dfs = [pd.read_csv(DATA_DIR + name, index_col='Unnamed: 0') for name in filenames]

In [4]:
df = pd.concat(dfs)
df = df.sample(frac=1)
df = df[['reply', 'comment_score', 'reply_score']]
df = df.reset_index(drop=True)
df.head()

,reply,comment_score,reply_score
0,I love this interpretation,23,2
1,Thanks for the insight. I have some questions ...,22,1
2,"Definitely, all characters should be flawed ju...",4,3
3,Anywhere really.\n\nOne of my personal values ...,178,10
4,"I know there is his book *On Writing*, which i...",1,2


In [5]:
def regex_text(text):
    text = html.unescape(text)
    text = re.sub(r"\\'", r"'", text)
    text = re.sub(r"\s+$", '', text)    
    return text

def clean_dataframe(df):
    df['reply'] = df['reply'].astype(str)
    df['reply'] = df['reply'].apply(regex_text)
    df = df[df['reply'].str.len() != 0]
    return df

df = clean_dataframe(df)
df.head()

,reply,comment_score,reply_score
0,I love this interpretation,23,2
1,Thanks for the insight. I have some questions ...,22,1
2,"Definitely, all characters should be flawed ju...",4,3
3,Anywhere really.\n\nOne of my personal values ...,178,10
4,"I know there is his book *On Writing*, which i...",1,2


In [6]:
def minmax_scale(X, X_min, X_max):
    X_scaled = (X - X_min) / (X_max - X_min)
    return X_scaled

#print(df['comment_score'].)

# Scaling is [MinMax -> np.exp -> MinMax] such that the comments/replies with a higher count have more influence
def scale(df, cols=['comment_score', 'reply_score']):
    min_score = df[cols].min().min()
    print("Min score: ", min_score)
    max_score = df[cols].max().max()
    print("Max score: ", max_score)
    for col in cols:
        df[col+"_scaled"] = df[col]
        col = col+"_scaled"
        df[col] = df[col].apply(minmax_scale, args=(min_score, max_score))
        df[col] = df[col].apply(np.exp)

    cols = ['comment_score_scaled', 'reply_score_scaled']
    min_score = df[cols].min().min()
    print("Min score: ", min_score)
    max_score = df[cols].max().max()
    print("Max score: ", max_score)
    for col in cols:
        df[col] = df[col].apply(minmax_scale, args=(min_score, max_score))
    return df

df = scale(df)
df['reply_score_minmax'] = df['reply_score'].apply(minmax_scale, args=(df['reply_score'].min(), df['reply_score'].max()))
df['score_ratio'] = df['reply_score_scaled']/df['comment_score_scaled']

Min score:  -119
Max score:  8401
Min score:  1.0
Max score:  2.718281828459045


In [7]:
len_df = len(df)
split_ratio = 0.9
train = df.iloc[:int(split_ratio*len_df)][['reply', 'reply_score_minmax']].rename(columns={'reply_score_minmax': 'label'})
validation = df.iloc[int(split_ratio*len_df):][['reply', 'reply_score_minmax']].rename(columns={'reply_score_minmax': 'label'})

dataset = dict()
dataset['validation'] = Dataset.from_pandas(validation, preserve_index=False)
dataset['train'] = Dataset.from_pandas(train, preserve_index=False)
datasets = DatasetDict(dataset)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["reply"], truncation=True, padding=True, max_length=MAX_LEN)

tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["reply"],
    )

# Training

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="../output/bert_predictor",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    save_steps=10000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

trainer.train()

/home/charlie/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 500/31215 [03:45<3:31:12,  2.42it/s]

{'loss': 0.0013, 'learning_rate': 1.967964119814192e-05, 'epoch': 0.02}


  3%|▎         | 1000/31215 [07:12<3:27:58,  2.42it/s]

{'loss': 0.0001, 'learning_rate': 1.935928239628384e-05, 'epoch': 0.03}


  5%|▍         | 1500/31215 [10:38<3:24:52,  2.42it/s]

...

 99%|█████████▉| 31000/31215 [3:39:44<01:25,  2.51it/s]

{'loss': 0.0, 'learning_rate': 1.3775428479897487e-07, 'epoch': 0.99}


                                                       
100%|██████████| 31215/31215 [3:49:34<00:00,  2.27it/s]

{'eval_loss': 6.665878026979044e-05, 'eval_runtime': 504.9992, 'eval_samples_per_second': 27.473, 'eval_steps_per_second': 6.869, 'epoch': 1.0}
{'train_runtime': 13774.7543, 'train_samples_per_second': 9.064, 'train_steps_per_second': 2.266, 'train_loss': 9.235609688274127e-05, 'epoch': 1.0}


TrainOutput(global_step=31215, training_loss=9.235609688274127e-05, metrics={'train_runtime': 13774.7543, 'train_samples_per_second': 9.064, 'train_steps_per_second': 2.266, 'train_loss': 9.235609688274127e-05, 'epoch': 1.0})

In [10]:
trainer.save_model('../output/bert_predictor/final')

In [24]:
import torch
test_input = tokenizer(validation['reply'].iloc[151], return_tensors='pt').to(device)
with torch.no_grad():
    output = model(**test_input)

output.logits[0]

tensor([0.0377], device='cuda:0')